# Digital Twin Data Acquisition

In this notebook, we will use Big Query and The Graph to obtain the signals required for the Rai Digital Twin. We will get state snapshots and calculate the absolute changes between time periods. 

# TODO: Define features below, add time frequency analysis - add readmes and document

### Signals  
* Price signals:
    * Eth price
    * Rai price in eth - need new source
    * Rai price in usd - need new source
    * Redemption price
    * Redemption rate.
* Uniswap data - https://thegraph.com/explorer/subgraph/uniswap/uniswap-v2
* Total debt = sum over SAFEs of debt
* Total collateral = sum over SAFEs of Collateral
* Total rai = directly observable from the RAI erc 20 contract (not sure how to get it from theGraph), further note that total rai = principle debt
* Excess Liability = Total debt - total rai
* ETH in Uniswap = should be queriable from thegraph data on uniswap (different source subgraph) --> future replace this with total value in ETH of assets RAI is paired with on Uniswap instances
* RAI in Uniswap = should be queriable from thegraph data on uniswap
* Debt ceiling = directly observable from RAI subgraph
* debt as fraction of debt ceiling = total debt/debt ceiling
* floating RAI = total rai - rai in uniswap
* floating RAI as fraction of total RAI = floating RAI /total RAI
another thing i am interested in from uniswap is the yield per unit liquidity
* total liquidity = price_usd_asset1 * quantity_of_asset1 + price_usd_asset2*quantity_of_asset2
* volume_per_period = take directly from contract
* fees_per_period = measure directly or compute from volumes
* revenues in token1 and token2
* value of profits in USD
* yields = profit in usd per total liquidity in USD of the whole pool
* Ratio of yields to total liquidity

In [1]:
# import libraries
import os
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import json
import requests
# The Graph URL
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

%matplotlib inline

# constants
constant = 1000000000000000000

#defining creditionals
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/aclarkdata/Downloads/raidata-5e1723d42cf5.json'

# initializing agent
client = bigquery.Client()


In [2]:
# SQL query
sql = """
SELECT * 
FROM `blockchain-etl.ethereum_rai.ChainlinkMedianETHUSD_event_UpdateResult`
ORDER By block_timestamp DESC

"""

eth_price_next_OSM = client.query(sql).to_dataframe()
eth_price_next_OSM['ETH Price (Next OSM)'] = eth_price_next_OSM['medianPrice'].astype(float)/constant
# subset
eth_price_next_OSM = eth_price_next_OSM[['block_number','ETH Price (Next OSM)']]

eth_price_next_OSM.head()

,block_number,ETH Price (Next OSM)
0,12271233,2129.458042
1,12270946,2219.590000
2,12270590,2244.796301
3,12270259,2236.288682
4,12269966,2218.569789


In [3]:
# SQL query
sql = """
SELECT 
* 
FROM `blockchain-etl.ethereum_rai.OSM_event_UpdateResult`
ORDER By block_timestamp DESC

"""

eth_price_OSM = client.query(sql).to_dataframe()
eth_price_OSM['ETH Price (OSM)'] = eth_price_OSM['newMedian'].astype(float)/constant
# subset
eth_price_OSM = eth_price_OSM[['block_number','ETH Price (OSM)']]

eth_price_OSM.head()

,block_number,ETH Price (OSM)
0,12343619,2765.230000
1,12343350,2759.543733
2,12343097,2745.469394
3,12342826,2749.848368
4,12342586,2733.914147


In [4]:
# blocknumbers = eth_price_OSM.block_number.values.tolist()


# uniswap_url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
# pair = []
# for i in blocknumbers[-5:-1]:
#     query = '''
#     {
#       pairs(block: {number:%s}, where: {id: "0x8ae720a71622e824f576b4a8c03031066548a3b1"}){
#         id,
#         token0 {
#           id
#         },
#         token1 {
#           id
#         }
#         reserve0,
#         reserve1,
#         totalSupply,
#         reserveETH,
#         reserveUSD,
#         token0Price,
#         token1Price
#       }
#     }
    
#     ''' % i
#     r = requests.post(uniswap_url, json = {'query':query})
#     s = json.loads(r.content)['data']['pairs'][0]
#     pair.append(s)
        
# pairState = pd.DataFrame(pair)

# #pairState['block_number'] = blocknumbers


# pairState.head()



In [5]:
blocknumbers = eth_price_OSM.block_number.values.tolist()

state = []
for i in blocknumbers[:-5]:
    query = '''
    {
      systemState(block: {number:%s},id:"current") { 
        coinUniswapPair {
          reserve0
          reserve1
        }
        currentCoinMedianizerUpdate{
          value
        }
        currentRedemptionRate {
          eightHourlyRate
          annualizedRate
          hourlyRate
          createdAt
        }
        currentRedemptionPrice {
          value
        }
        erc20CoinTotalSupply
        globalDebt
        globalDebtCeiling
        safeCount,
        totalActiveSafeCount
        coinAddress
        wethAddress
        systemSurplus
        debtAvailableToSettle
        lastPeriodicUpdate
        createdAt
        createdAtBlock
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['systemState']
    state.append(s)
        
systemState = pd.DataFrame(state)

systemState['block_number'] = blocknumbers[:-5]


systemState.head()


,coinAddress,coinUniswapPair,createdAt,createdAtBlock,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,lastPeriodicUpdate,safeCount,systemSurplus,totalActiveSafeCount,wethAddress,block_number
0,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '18760441.619630650840377541', 'r...",1529678381,5834717,{'value': '3.026441279204366679'},{'value': '3.013174389557087774776246586'},{'annualizedRate': '0.959091688469505652887305...,110222.072759318338351381395940064,29889930.145013977642111794,30298592.34210970579073454463023908,115792089237316195423570985008687.9,1619809463,1947,224667.1525360347250503315345315396,629,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12343619
1,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '18761033.027017254827017699', 'r...",1529678381,5834717,{'value': '3.026441279204366679'},{'value': '3.013188700850922842843182689'},{'annualizedRate': '0.959091688469505652887305...,110222.072759318338351381395940064,29901219.077161608088281094,30309626.74151735819732442115094642,115792089237316195423570985008687.9,1619797135,1947,224559.8576606969883119130185310233,630,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12343350
2,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '18761202.187824832515602319', 'r...",1529678381,5834717,{'value': '3.026441279204366679'},{'value': '3.013209195610230437197127559'},{'annualizedRate': '0.919604743607408732957270...,110222.072759318338351381395940064,29902021.462911729191452126,30310425.82418890349437257115094642,115792089237316195423570985008687.9,1619797135,1947,224559.8576606969883119130185310233,631,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12343097
3,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '18746354.646033961753595759', 'r...",1529678381,5834717,{'value': '3.026441279204366679'},{'value': '3.013264964009561820661937986'},{'annualizedRate': '0.919604743607408732957270...,110222.072759318338351381395940064,29902824.441218692393115718,30311225.49697798808600947915094642,115792089237316195423570985008687.9,1619797135,1947,224559.8576606969883119130185310233,632,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12342826
4,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '18762169.698592691451007785', 'r...",1529678381,5834717,{'value': '3.039897627615952124'},{'value': '3.013264964009561820661937986'},{'annualizedRate': '0.919604743607408732957270...,110222.072759318338351381395940064,29902824.441218692393115718,30311788.24432448720414338725354107,115792089237316195423570985008687.9,1619787081,1947,224454.9283691419108082186449205912,632,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12342586


In [6]:
systemState.tail()

,coinAddress,coinUniswapPair,createdAt,createdAtBlock,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,lastPeriodicUpdate,safeCount,systemSurplus,totalActiveSafeCount,wethAddress,block_number
1777,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",1529678381,5834717,{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.696064173575505967822069085347,115792089237316195423570985008687.9,1613409220,3,0.1278156009542366213422373102764649,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11862898
1778,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",1529678381,5834717,{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.696064173575505967822069085347,115792089237316195423570985008687.9,1613405255,3,0.1278156009542366213422373102764649,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11862600
1779,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",1529678381,5834717,{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.696064173575505967822069085347,115792089237316195423570985008687.9,1613405255,3,0.1278156009542366213422373102764649,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11862330
1780,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",1529678381,5834717,{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,1613394096,3,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11862059
1781,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",1529678381,5834717,{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,1613394096,3,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861791


In [7]:
systemState.coinUniswapPair.values[0]

{'reserve0': '18760441.619630650840377541',
 'reserve1': '20575.080688180122649403'}

In [8]:
systemState.currentCoinMedianizerUpdate.values[0]

{'value': '3.026441279204366679'}

In [9]:
systemState['RedemptionRateAnnualizedRate'] = systemState.currentRedemptionRate.apply(lambda x: x['annualizedRate'])    
systemState['RedemptionRateHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['hourlyRate'])
systemState['RedemptionRateEightHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['eightHourlyRate'])
systemState['RedemptionPrice'] = systemState.currentRedemptionPrice.apply(lambda x: x['value'])
systemState['RAIInUniswapV2(RAI/ETH)'] = systemState.coinUniswapPair.apply(lambda x: x['reserve0'])
del systemState['currentRedemptionRate']
del systemState['currentRedemptionPrice']
systemState['RedemptionRateAnnualizedRate'] = systemState['RedemptionRateAnnualizedRate'].astype(float)
systemState['RedemptionRateHourlyRate'] = systemState['RedemptionRateHourlyRate'].astype(float)
systemState['RedemptionRateEightHourlyRate'] = systemState['RedemptionRateEightHourlyRate'].astype(float)
systemState['RedemptionPrice'] = systemState['RedemptionPrice'].astype(float)
systemState['RAIInUniswapV2(RAI/ETH)'] = systemState['RAIInUniswapV2(RAI/ETH)'].astype(float)


# subset
systemState = systemState[['debtAvailableToSettle','erc20CoinTotalSupply','globalDebt',
                           'globalDebtCeiling','systemSurplus','totalActiveSafeCount',
                           'block_number', 'RedemptionRateAnnualizedRate',
                           'RedemptionRateHourlyRate', 'RedemptionRateEightHourlyRate',
                        'RedemptionPrice', 'RAIInUniswapV2(RAI/ETH)']]

In [10]:
#       hourlyStats(block: {number:%s}) { 


hourly = []
for i in blocknumbers[1:-5]:
    query = '''
    {
      hourlyStats(where: {blockNumber_gt: %s}) { 
        marketPriceUsd # price of COIN in USD (uni pool price * ETH median price)
        marketPriceEth # Price of COIN in ETH (uni pool price)
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats'][0]
    hourly.append(s)
        
hourlyStats = pd.DataFrame(hourly)

hourlyStats['block_number'] = blocknumbers[1:-5]

hourlyStats.head()


,marketPriceEth,marketPriceUsd,block_number
0,0.001096726884438085941097668294104614,3.038342192760632322028733498776913,12343350
1,0.001096726884438085941097668294104614,3.038342192760632322028733498776913,12343097
2,0.001096726884438085941097668294104614,3.038342192760632322028733498776913,12342826
3,0.001096849108212519728430234065751126,3.016168729809461057763676034466064,12342586
4,0.001096849108212519728430234065751126,3.016168729809461057763676034466064,12342308


In [11]:
hourlyStats.describe()

,block_number
count,1.781000e+03
mean,1.210268e+07
std,1.391693e+05
min,1.186179e+07
25%,1.198232e+07
50%,1.210255e+07
75%,1.222326e+07
max,1.234335e+07


In [12]:
safehistories = []
for i in blocknumbers[:-5]:
    query = '''
    {
      safes(block: {number:%s}) {
            collateral
            debt
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['safes']
    t = pd.DataFrame(s)
    t['collateral'] = t['collateral'].astype(float)
    t['debt'] = t['debt'].astype(float)
    safehistories.append(pd.DataFrame(t.sum().to_dict(),index=[0]))

safe_history = pd.concat(safehistories)


In [13]:
safe_history.head()

,collateral,debt
0,1482.8032,327195.92226
0,1482.8032,327195.92226
0,1482.8032,327195.92226
0,1482.8032,327195.92226
0,1482.8032,327195.92226


In [14]:
safe_history['block_number'] = blocknumbers[:-5]
safe_history.reset_index(inplace=True)
del safe_history['index']

In [15]:
states = hourlyStats.merge(systemState,how='inner',on='block_number')
states = states.merge(safe_history,how='inner',on='block_number')

In [16]:
# prices = .merge(eth_price_next_OSM,how='inner',on='block_number')
# prices.head()

In [17]:
states = states.merge(eth_price_OSM,how='inner',on='block_number')

In [18]:
states.head()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RAIInUniswapV2(RAI/ETH),collateral,debt,ETH Price (OSM)
0,0.001096726884438085941097668294104614,3.038342192760632322028733498776913,12343350,110222.072759318338351381395940064,29901219.077161608088281094,30309626.74151735819732442115094642,115792089237316195423570985008687.9,224559.8576606969883119130185310233,630,0.959092,0.999995,0.999962,3.013189,1.876103e+07,1482.8032,327195.92226,2759.543733
1,0.001096726884438085941097668294104614,3.038342192760632322028733498776913,12343097,110222.072759318338351381395940064,29902021.462911729191452126,30310425.82418890349437257115094642,115792089237316195423570985008687.9,224559.8576606969883119130185310233,631,0.919605,0.999990,0.999923,3.013209,1.876120e+07,1482.8032,327195.92226,2745.469394
2,0.001096726884438085941097668294104614,3.038342192760632322028733498776913,12342826,110222.072759318338351381395940064,29902824.441218692393115718,30311225.49697798808600947915094642,115792089237316195423570985008687.9,224559.8576606969883119130185310233,632,0.919605,0.999990,0.999923,3.013265,1.874635e+07,1482.8032,327195.92226,2749.848368
3,0.001096849108212519728430234065751126,3.016168729809461057763676034466064,12342586,110222.072759318338351381395940064,29902824.441218692393115718,30311788.24432448720414338725354107,115792089237316195423570985008687.9,224454.9283691419108082186449205912,632,0.919605,0.999990,0.999923,3.013265,1.876217e+07,1482.8032,327195.92226,2733.914147
4,0.001096849108212519728430234065751126,3.016168729809461057763676034466064,12342308,110222.072759318338351381395940064,29905327.453351736217323311,30314280.96994956820154385625354107,115792089237316195423570985008687.9,224454.9283691419108082186449205912,633,0.919605,0.999990,0.999923,3.013296,1.875978e+07,1482.8032,327195.92226,2751.484809


In [19]:
states.tail()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RAIInUniswapV2(RAI/ETH),collateral,debt,ETH Price (OSM)
1776,0.001768934955701819449591545208406114,3.212255638445860905952903939205175,11862898,0,3048.440432971667032725,3048.696064173575505967822069085347,115792089237316195423570985008687.9,0.1278156009542366213422373102764649,3,1.0,1.0,1.0,3.14,801.933727,16.89236,3048.35849,1825.887144
1777,0.001768934955701819449591545208406114,3.223317461414449337259100162066151,11862600,0,3048.440432971667032725,3048.696064173575505967822069085347,115792089237316195423570985008687.9,0.1278156009542366213422373102764649,3,1.0,1.0,1.0,3.14,801.933727,16.89236,3048.35849,1793.770282
1778,0.001768934955701819449591545208406114,3.223317461414449337259100162066151,11862330,0,3048.440432971667032725,3048.696064173575505967822069085347,115792089237316195423570985008687.9,0.1278156009542366213422373102764649,3,1.0,1.0,1.0,3.14,801.933727,16.89236,3048.35849,1803.891149
1779,0.001768934955701819449591545208406114,3.215446177235759811208395560891868,11862059,0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,1.0,1.0,1.0,3.14,801.933727,16.89236,3048.35849,1803.891149
1780,0.001768934955701819449591545208406114,3.215446177235759811208395560891868,11861791,0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,1.0,1.0,1.0,3.14,801.933727,16.89236,3048.35849,1805.792735


In [20]:
states = states.astype(float)

In [21]:
# export
states.to_csv('states.csv')